Multi-platform loading

In [ ]:

###SETTINGS###

import os
import subprocess
import shutil
from getpass import getpass

# Platform detection
ON_KAGGLE = os.path.exists('/kaggle')
ON_COLAB = 'COLAB_RELEASE_TAG' in os.environ or os.path.exists('/content')

if shutil.which("aria2c") is None:
    print("aria2c not found → installing...")
    try:
        subprocess.run(["apt", "update", "-qq"], check=True, capture_output=True)
        result = subprocess.run(["apt", "install", "-y", "-qq", "aria2"], capture_output=True, text=True)
        if result.returncode == 0:
            print("aria2c installed successfully")
        else:
            print("Install failed (code {}):".format(result.returncode))
            print("stderr:", result.stderr.strip())
    except subprocess.CalledProcessError as e:
        print(f"apt error (code {e.returncode}): {e.stderr}")
    except Exception as e:
        print(f"Unexpected error: {e}")
else:
    print("aria2c already available")

# Determining the working directory
possible_bases = [
    "/workspace",       # Vast.ai / RunPod
    "/kaggle/working",  # Kaggle
    "/content",         # Google Colab
]

BASE_DIR = None
for path in possible_bases:
    if os.path.isdir(path):
        BASE_DIR = path
        break

if BASE_DIR is None:
    BASE_DIR = os.getcwd()
    print("WARNING: Known directory not found:", BASE_DIR)

print("Working directory:", BASE_DIR)

# Configuration
FORGE_DIR        = os.path.join(BASE_DIR, "stable-diffusion-webui-forge")
MODELS_DIR       = os.path.join(BASE_DIR, "stable-diffusion-webui", "models", "Stable-diffusion")
LORA_DIR         = os.path.join(BASE_DIR, "stable-diffusion-webui", "models", "Lora")
CONTROLNET_DIR   = os.path.join(BASE_DIR, "stable-diffusion-webui", "models", "ControlNet")
EXTENSIONS_DIR   = os.path.join(BASE_DIR, "stable-diffusion-webui", "extensions")
OUTPUTS_DIR      = os.path.join(BASE_DIR, "stable-diffusion-webui", "outputs")

for d in [MODELS_DIR, LORA_DIR, CONTROLNET_DIR, EXTENSIONS_DIR, OUTPUTS_DIR]:
    os.makedirs(d, exist_ok=True)


def get_secret(name: str):
    """Get secret from env/Kaggle/Colab, then optional prompt."""
    value = os.environ.get(name)
    if value:
        return value.strip()

    # Kaggle secrets
    if ON_KAGGLE:
        try:
            from kaggle_secrets import UserSecretsClient
            value = UserSecretsClient().get_secret(name)
            return value.strip() if value else None
        except Exception:
            pass

    # Colab secrets panel: from google.colab import userdata
    if ON_COLAB:
        try:
            from google.colab import userdata
            value = userdata.get(name)
            if value:
                return value.strip()
        except Exception:
            pass

    # Manual fallback for Vast.ai/local
    try:
        return getpass(f"{name}: ").strip() or None
    except Exception:
        return None


CIVITAI_TOKEN = get_secret("CIVITAI_TOKEN")
HF_TOKEN = get_secret("HF_TOKEN")

TOKENS = {}
if CIVITAI_TOKEN:
    TOKENS["CIVITAI"] = CIVITAI_TOKEN
if HF_TOKEN:
    TOKENS["HF"] = HF_TOKEN

if not CIVITAI_TOKEN:
    print("CivitAI token not found")
if not HF_TOKEN:
    print("HF token not found")


In [ ]:

# Download function
from urllib.parse import urlencode


def _prepare_download_url(url, token):
    """CivitAI download works more reliably with token as query param."""
    if token and "civitai.com/api/download/models" in url and "token=" not in url:
        sep = "&" if "?" in url else "?"
        return f"{url}{sep}{urlencode({'token': token})}"
    return url


def _run_cmd(cmd):
    return subprocess.run(cmd, check=True, capture_output=True, text=True)


def download_file(url, output_path, token=None):
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    if os.path.exists(output_path) and os.path.getsize(output_path) > 1_000_000:
        return

    token = token.strip() if isinstance(token, str) else token
    final_url = _prepare_download_url(url, token)

    # Prefer aria2c for speed, but retry with wget/curl for compatibility.
    attempts = []

    if shutil.which("aria2c"):
        aria2_cmd = [
            "aria2c", "-x", "16", "-s", "16", "-k", "1M",
            "--summary-interval=0", "--console-log-level=error",
            "-d", os.path.dirname(output_path),
            "-o", os.path.basename(output_path),
            "--max-tries=3", "--retry-wait=2",
            final_url,
        ]
        if token:
            aria2_cmd += ["--header", f"Authorization: Bearer {token}"]
        attempts.append(("aria2c", aria2_cmd))

    wget_cmd = [
        "wget", "-c", "-q", final_url, "-O", output_path,
        "--tries=3", "--waitretry=2",
        "--user-agent=Mozilla/5.0",
    ]
    if token:
        wget_cmd = wget_cmd[:3] + ["--header", f"Authorization: Bearer {token}"] + wget_cmd[3:]
    attempts.append(("wget", wget_cmd))

    if shutil.which("curl"):
        curl_cmd = [
            "curl", "-L", "-C", "-", "--fail", "--silent", "--show-error",
            "--retry", "3", "--retry-delay", "2",
            "-A", "Mozilla/5.0",
            final_url, "-o", output_path,
        ]
        if token:
            curl_cmd = curl_cmd[:1] + ["-H", f"Authorization: Bearer {token}"] + curl_cmd[1:]
        attempts.append(("curl", curl_cmd))

    last_error = None
    for tool_name, cmd in attempts:
        try:
            _run_cmd(cmd)
            if os.path.exists(output_path) and os.path.getsize(output_path) > 1_000_000:
                print(f"  OK → {os.path.basename(output_path)} ({tool_name})")
                return
            print(f"  ERROR → {os.path.basename(output_path)} ({tool_name} produced small file)")
        except subprocess.CalledProcessError as e:
            last_error = e
            err = (e.stderr or "").strip()
            short_err = err.splitlines()[-1] if err else "no stderr"
            print(f"  ERROR → {os.path.basename(output_path)} ({tool_name}, code {e.returncode})")
            print(f"    {short_err}")

    if last_error:
        raise last_error
    raise RuntimeError("No downloader available")


In [ ]:
lora_to_download = [
    ("Detailer IL V2 218 MB",        "https://civitai.com/api/download/models/1736373?type=Model&format=SafeTensor",    "detailer_v2_il.safetensors",     "CIVITAI"),
    ("Realistic filter V1 55 MB",    "https://civitai.com/api/download/models/1124771?type=Model&format=SafeTensor",    "realistic_filter_v1_il.safetensors", "CIVITAI"),
]

print(f"Total LoRa in the list: {len(lora_to_download)}")
skipped = 0

print(f"All LoRA: {len(lora_to_download)}")
skipped = 0

for name, url, filename, source in lora_to_download:
    output_path = os.path.join(LORA_DIR, filename)
    
    if os.path.exists(output_path) and os.path.getsize(output_path) > 1_000_000:
        print(f"  Уже есть → {name:<30} {filename}")
        skipped += 1
        continue
    
    token = TOKENS.get(source)  # CIVITAI → CIVITAI_TOKEN value, HF → HF_TOKEN value
    
    print(f"  Loading {name:<30} ({source}) → {filename}")
    try:
        download_file(url=url, output_path=output_path, token=token)
    except Exception as e:
        print(f"  ERROR: {e}")

print(f"\nDONE. New: {len(lora_to_download) - skipped}, skipped: {skipped}")